In [1]:
import pandas as pd
import math
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import geopy as gp
import plotly.graph_objects as go
import plotly.express as px

plt.style.use("ggplot")

In [2]:
upwork = pd.read_csv("../datasets/upwork_dataScience.csv")
upwork.head()

,Title,Description,Skills,Source Country,Client Porpouses,Average Hour Paid ($),Total Spent (K$),Domain of Job,Fixed Price ($),Price per Hour ($),Work on,Experience Required
0,Webcrawler to collect data of all escaperooms ...,For a specific project we need as much informa...,"['Data Scraping', 'Data Mining', 'Data Extract...",Netherlands,53,20.00,6.2,NaN,300.0,NaN,Remote Job,Expert
1,Build a Browse AI Robot to Multiple Sites to E...,Im looking to use the Browse AI software to ex...,"['Artificial Intelligence', 'Data Scraping', '...",United States,80,8.38,43.0,"['Sales', 'Marketing']",NaN,NaN,Remote Job,Intermediate
2,"Google Analytics, GA4 Migration, and Set-up",We are seeking an experienced Google Analytics...,"['Marketing Analytics', 'Operations Analytics'...",United States,15,46.82,6.4,NaN,NaN,"['60.00', '80.00']",Remote Job,Expert
3,Google Trends API - Python Notebook,Build a Google Collab Notebook that queries th...,"['API', 'Python', 'Jupyter', 'Google Trends']",Australia,117,16.70,393.0,NaN,200.0,NaN,Remote Job,Intermediate
4,Data analysis in excel,I have a spreadsheet of raw data from a labor ...,"['Microsoft Excel', 'Data Analysis']",United States,4,17.68,818.0,NaN,NaN,"['5.00', '20.00']",Remote Job,Entry level


In [3]:
skills = []

for row in upwork['Skills']:
    row = eval(row)
    row = [item.replace('Outra - ', '') for item in row]
    skills += row

skills = pd.DataFrame({
    "Skills": skills
})
skills.head()

skills = skills.groupby(by='Skills').size().reset_index().rename(columns={0:"Size"}).sort_values(by="Size", ascending=False)

In [5]:
skills = {skill:(i + 1) for i, skill in enumerate(skills['Skills'])}

skills['None'] = 0

users = {}

for i, row in enumerate(upwork['Skills']):
    v_vector = eval(row)
    v_vector = [skills[value] for value in v_vector]
    users[i] = v_vector

sizes = [len(list(users.values())[i]) for i in users.keys()]

sizes = np.sort(sizes)
users_idx = [i for i in range(len(sizes))]

fig = go.Figure([go.Scatter(
    x=users_idx,
    y=sizes,
    mode="markers+lines"
)])

fig.update_layout(
    title="Distribuição de quantidade de habilidades por proposta",
    xaxis_title="Propostas",
    yaxis_title="# de habilidades",
    font=dict(
        size=16
    )
)

fig.show()

In [7]:
porpouses_graph = {}

for user, u_skills in users.items():
    user_edges = []
    for user_2, u_skills_2 in users.items():
        if user == user_2: continue
        similarity_count = 0
        for s in u_skills:
            if s in u_skills_2:
                similarity_count += 1
                if similarity_count >= 3:
                    user_edges.append(user_2)
                    break

    porpouses_graph[user] = user_edges

sizes = [len(list(porpouses_graph.values())[i]) for i in porpouses_graph.keys()]

sizes = np.sort(sizes)
users_idx = [i for i in range(len(sizes))]

fig = go.Figure([go.Scatter(
    x=users_idx,
    y=sizes,
    mode="markers+lines"
)])

fig.update_layout(
    title="Distribuição de conexões por serviço oferecido",
    xaxis_title="User ID",
    yaxis_title="# de conexões",
    font=dict(
        size=16
    )
)

fig.show()

### Criando grafo com threashold de 3

In [8]:
print("Criando Grafo...")
graph = nx.from_dict_of_lists(porpouses_graph)

# Obter as posições dos nós usando o algoritmo de layout spring
print("Obtendo posições dos nós...")
pos = nx.spring_layout(graph)

# Criar lista de posições x e y para os nós
x = [pos[node][0] for node in graph.nodes()]
y = [pos[node][1] for node in graph.nodes()]

print("Criando lista de arestas...")
# Criar lista de arestas
edge_trace = go.Scatter(
    x=[],
    y=[],
    line=dict(width=0.5, color='#888'),
    hoverinfo='none',
    mode='lines')

print("Adicionar arestas ao gráfico")
# Adicionar as arestas ao gráfico
for edge in graph.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_trace['x'] += tuple([x0, x1, None])
    edge_trace['y'] += tuple([y0, y1, None])

print("Criar lista de nós...")
# Criar lista de nós
node_trace = go.Scatter(
    x=x,
    y=y,
    text=list(graph.nodes()),
    mode='markers+text',
    hoverinfo='text',
    marker=dict(
        showscale=False,
        colorscale='YlGnBu',
        reversescale=True,
        color=[],
        size=10,
        colorbar=dict(
            thickness=15,
            title='Node Connections',
            xanchor='left',
            titleside='right'
        ),
        line_width=2))

# Adicionar a cor dos nós baseada no número de conexões
# node_trace.marker.color = [len(graph.neighbors(node)) for node in graph.nodes()]

print("Criar layout do gráfico...")
# Criar layout do gráfico
layout = go.Layout(
    title='Grafo',
    showlegend=False,
    hovermode='closest',
    margin=dict(b=20, l=5, r=5, t=40),
    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))

# Criar figura do Plotly
fig = go.Figure(data=[edge_trace, node_trace], layout=layout)

# Exibir o gráfico
fig.show()

Criando Grafo...
Obtendo posições dos nós...
Criando lista de arestas...
Adicionar arestas ao gráfico


KeyboardInterrupt: 

## O que conseguimos concluir baseando nesso Grafo de propostas?

- Consigo plotar um mapa dos paises que fornecem propostas? (DONE)

- Quais os domínios que mais recebem propostas? (DONE)

- Quanto é pago em média por cada domínio? (DONE)

- Quais skills são mais bem pagas em média? (DONE)

- Quais países estão mais dispostos a pagar por serviços de data science? (DONE)

- Qual a média de propostas por país? (DONE)

- Média de preço por hora. Média de preço fixo. (DONE)

- Distribuição de requerimentos de experiencia. (DONE)

### Skills mais bem valorizadas

In [3]:
skills = upwork[['Skills', 'Fixed Price ($)']].copy().dropna()

In [4]:
skills['Skills'] = skills['Skills'].apply(lambda x: eval(x))

In [5]:
skills_paid = []

for i, skill_list in enumerate(skills['Skills']):
    for skill in skill_list:
         skills_paid.append((skill, skills.iloc[i, 1]))

In [6]:
skills_paid = pd.DataFrame(skills_paid, columns=["Skill", "Value ($)"])
skills_paid.head()

,Skill,Value ($)
0,Data Scraping,300.0
1,Data Mining,300.0
2,Data Extraction,300.0
3,Company Research,300.0
4,Data Extraction,300.0


In [8]:
skills_paid["Value ($)"] = skills_paid["Value ($)"].apply(lambda row: float(row))

In [9]:
skill_size = skills_paid['Skill'].value_counts().reset_index()
skill_size.head()

,index,Skill
0,Python,132
1,Data Scraping,76
2,Data Analysis,67
3,Machine Learning,67
4,Microsoft Excel,66


In [10]:
mean_sp = skills_paid.groupby(by="Skill")['Value ($)'].mean().reset_index().sort_values(by='Value ($)', ascending=False)
mean_sp.head()

,Skill,Value ($)
195,HTML5,22750.0
392,Website,22750.0
63,CSS 3,22750.0
295,Prototyping,22750.0
4,AI-Generated Video,20000.0


In [12]:
count = []

for skill in mean_sp['Skill']:
    c = skill_size.loc[skill_size['index'] == skill]['Skill'].values[0]
    count.append(c)

In [13]:
mean_sp['Count'] = count
mean_sp.head()
mean_sp = mean_sp.sort_values(by="Count", ascending=False).head(10)

In [14]:
# Criação do gráfico de dispersão 3D
fig = go.Figure(data=go.Scatter3d(
    x=mean_sp['Skill'],
    y=mean_sp['Value ($)'],
    z=mean_sp['Count'],
    mode='markers',
    marker=dict(
        size=10,
        color=mean_sp['Count'],
        colorscale='Viridis',
        opacity=0.8
    )
))

# Configurações do layout
fig.update_layout(
    scene=dict(
        xaxis=dict(
            title='Habilidade',
            tickmode='linear',
            tick0=0,
            dtick=1
        ),
        yaxis_title='Valor Pago ($)',
        zaxis_title='Quantidade de Propostas'
    ),
    width=800,  # Largura da figura
    height=600  # Altura da figura
)

# Exibição do gráfico
fig.show()

### Média de preços por hora e preços fixos

In [15]:
fixed_prices_mean = upwork['Fixed Price ($)'].dropna().mean()
print(f"Fixed Price Mean: {fixed_prices_mean:.2f}$")

Fixed Price Mean: 810.09$


In [16]:
hour_price = upwork['Price per Hour ($)'].dropna().values
hour_price = [eval(x) for x in hour_price]
min_price = [hour_price[i][0] for i in range(len(hour_price))]
max_price = [hour_price[i][1] for i in range(len(hour_price))]

min_price = np.array([float(x) for x in min_price]).mean()
max_price = np.array([float(x) for x in max_price]).mean()

print(f"Mean of Min Payment per hour: {min_price:.2f}$")
print(f"Mean of Max Payment per hour: {max_price:.2f}$")

Mean of Min Payment per hour: 22.43$
Mean of Max Payment per hour: 56.17$


### Domínios que mais recebem propostas

In [14]:
def concat_domains(texts: list):
    if len(texts) == 1: return texts[0]
    else: return " & ".join(texts)

def plot_graphics(data, title, x_label, y_label):
    fig = px.bar(data, x=x_label, y=y_label, title=title)

    fig.update_layout(
        font=dict(
            size=16  # Defina o tamanho da fonte desejado
        ),
        annotations=[
            go.layout.Annotation(
                x=xi,
                y=yi,
                text=str(yi),
                showarrow=False,
                font=dict(size=12)
            ) for xi, yi in zip(data[x_label], data[y_label])
            ]
        )

    fig.show()

In [18]:
domains = upwork['Domain of Job'].dropna().reset_index()
domains = domains.groupby(by="Domain of Job").size().reset_index()
domains['Domain of Job'] = domains['Domain of Job'].apply(lambda row: eval(row))
domains['Domain of Job'] = [concat_domains(x) for x in domains['Domain of Job'].values]

In [19]:
domains = domains.sort_values(by=0, ascending=False).rename(columns={0:"# of Jobs"})
domains.head()

,Domain of Job,# of Jobs
22,Tech & IT,135
18,Sales & Marketing,44
8,Finance & Accounting,26
4,Education,21
6,Engineering & Architecture,19


In [70]:
plot_graphics(domains.head(10), "Propostas por Domínio", "Domain of Job", "# of Jobs")
# Gráfico de Pizza

### Dominios mais bem pagos por preço fixo

In [21]:
paid_per_domain = upwork[['Domain of Job', 'Fixed Price ($)']]
paid_per_domain = paid_per_domain.dropna()
paid_per_domain = paid_per_domain.groupby(by="Domain of Job")['Fixed Price ($)'].mean().reset_index().sort_values(by="Fixed Price ($)", ascending=False)

In [22]:
paid_per_domain['Domain of Job'] = paid_per_domain['Domain of Job'].apply(lambda row: concat_domains(eval(row)))
paid_per_domain['Fixed Price ($)'] = paid_per_domain['Fixed Price ($)'].apply(lambda row: round(row, 2))

In [65]:
plot_graphics(paid_per_domain.head(10), 
              "Domínios mais bem pagos",
              "Domain of Job", "Fixed Price ($)")

### Gráfico 3D: Preço x Quantidade x Domínio

In [24]:
domain_filter = paid_per_domain['Domain of Job'].values
domains = domains.loc[domains["Domain of Job"].isin(domain_filter)]

In [25]:
d, p, q = [], [], []

for domain in domains['Domain of Job']:
    d.append(domain)
    p.append(paid_per_domain.loc[paid_per_domain["Domain of Job"] == domain]['Fixed Price ($)'].values[0])
    q.append(domains.loc[domains["Domain of Job"] == domain]['# of Jobs'].values[0])

In [26]:
graph3d = pd.DataFrame({
    'Domain of Job': d,
    '# of Jobs': q,
    'Fixed Price ($)': p
})

In [63]:
# Criação do gráfico de bolhas
fig = go.Figure(data=go.Scatter(
    x=graph3d['Domain of Job'],
    y=graph3d['Fixed Price ($)'],
    mode='markers',
    marker=dict(
        size=(graph3d['# of Jobs']*20),
        sizemode='diameter',
        sizeref=20,  # Controla o tamanho das bolhas
        sizemin=3
    )
))

# Configuração dos rótulos dos eixos
fig.update_layout(
    xaxis_title="Domínio",
    yaxis_title="Preço Fixo ($)",
    # annotations=[
    #     go.layout.Annotation(
    #         x=xi,
    #         y=yi,
    #         text=str(yi),
    #         showarrow=False,
    #         font=dict(size=12)
    #     ) for xi, yi in zip(graph3d['Domain of Job'], graph3d['# of Jobs']*20)
    #     ]
)

# Exibição do gráfico
fig.show()

### Dominios mais bem pagos por preços por hora

In [28]:
hour_per_domain = upwork[['Domain of Job', 'Price per Hour ($)']].dropna()
hour_per_domain['Domain of Job'] = hour_per_domain['Domain of Job'].apply(lambda row: concat_domains(eval(row)))
hour_per_domain['Price per Hour ($)'] = hour_per_domain['Price per Hour ($)'].apply(lambda row: eval(row))
hour_per_domain

,Domain of Job,Price per Hour ($)
15,Health & Fitness,"[10.00, 35.00]"
18,Tech & IT,"[50.00, 150.00]"
21,Tech & IT,"[35.00, 60.00]"
29,Tech & IT,"[18.00, 50.00]"
30,Tech & IT,"[18.00, 50.00]"
...,...,...
864,Media & Entertainment,"[5.00, 10.00]"
871,Tech & IT,"[5.00, 15.00]"
875,Energy & Utilities,"[10.00, 25.00]"
876,Tech & IT,"[40.00, 80.00]"


In [29]:
prices = np.array(hour_per_domain['Price per Hour ($)'].values)
hour_per_domain['min_prices'] = [x[0] for x in prices]
hour_per_domain['max_prices'] = [x[1] for x in prices]

In [30]:
hour_per_domain.drop('Price per Hour ($)', axis=1, inplace=True)

In [31]:
hour_per_domain['min_prices'] = hour_per_domain['min_prices'].apply(lambda x: float(x))
hour_per_domain['max_prices'] = hour_per_domain['max_prices'].apply(lambda x: float(x))

In [32]:
hour_per_domain = hour_per_domain.groupby(by="Domain of Job")[['min_prices', 'max_prices']].mean().reset_index()

In [33]:
fig = px.box(data_frame=hour_per_domain, x='Domain of Job', y=['min_prices', 'max_prices'])

fig.update_layout(
    title='Valores mínimos e máximos pagos por hora para cada Domínio',
    xaxis_title='Domínio',
    yaxis_title='Valores ($)',
    boxmode='group'  # Para agrupar os box plots lado a lado
)

fig.show()

### Quão os paises estão dispostos a pagar por serviços?

In [34]:
countries = list(set(upwork['Source Country'].dropna().values))
location = {}

from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="my_app")

def obter_coordenadas(nome_pais):
    location = geolocator.geocode(nome_pais)
    if location:
        latitude = location.latitude
        longitude = location.longitude
        return latitude, longitude
    else:
        return None

for country in countries:
    la, lo = obter_coordenadas(country)
    location[country] = (la, lo)

#### Mapa de propostas por paises

In [101]:
countries = upwork['Source Country'].dropna().reset_index()
countries = countries.groupby('Source Country').size().reset_index().rename(columns={0:"# of Jobs"}).sort_values('# of Jobs', ascending=False)

In [114]:
fig = px.bar(countries.head(10), y="Source Country", x="# of Jobs", title="Quais países contratam mais?", orientation='h')

fig.update_layout(
        font=dict(
            size=16  # Defina o tamanho da fonte desejado
        ),
        annotations=[
            go.layout.Annotation(
                x=xi,
                y=yi,
                text=str(yi),
                showarrow=False,
                font=dict(size=12)
            ) for xi, yi in zip(countries["# of Jobs"].head(10), countries["Source Country"].head(10))
            ]
        )

fig.show()

In [37]:
number_jobs = []

for country in location.keys():
    job = countries.loc[countries["Source Country"] == country]["# of Jobs"].values[0]
    number_jobs.append(job)

lat = [x[0] for x in location.values()]
lon = [x[1] for x in location.values()]

In [38]:
# Criação do mapa
fig = go.Figure(data=go.Scattergeo(
    lat=lat,
    lon=lon,
    mode='markers',
    marker=dict(
        size=number_jobs,
        color='red',
        line=dict(
            width=2,
            color='white'
        )
    )
))

# Configuração do layout do mapa
fig.update_layout(
    geo=dict(
        showland=True,
        landcolor='lightgray',
        countrycolor='white',
        coastlinecolor='white',
        projection_type='equirectangular'
    )
)

# Exibição do mapa
fig.show()

#### Mapa de pagamento por países

In [39]:
countries = upwork[['Source Country', 'Fixed Price ($)']].dropna()
countries = countries.groupby(by='Source Country')['Fixed Price ($)'].mean().reset_index().sort_values(by='Fixed Price ($)', ascending=False).head(10)
countries['Fixed Price ($)'] = countries['Fixed Price ($)'].apply(lambda x: round(x, 2))
# Em média, esses paises fazem propostas de projetos maiores para freelancers. E outros paises propoem projetos menores.

In [40]:
# key_countries = countries['Source Country'].values

# location = {k:v for k,v in location.items() if k in key_countries}

lat = [x[0] for x in location.values()]
lon = [x[1] for x in location.values()]

# value_jobs = []

# for country in location.keys():
#     job = countries.loc[countries["Source Country"] == country]['Fixed Price ($)'].values[0]
#     value_jobs.append(job)

In [41]:
plot_graphics(countries, 
              "Quais países pagam mais para freelancers",
              "Source Country", "Fixed Price ($)")

In [42]:
# Criação do mapa
fig = go.Figure(data=go.Scattergeo(
    lat=lat,
    lon=lon,
    mode='markers',
    marker=dict(
        size=10,
        color='red',
        line=dict(
            width=2,
            color='white'
        )
    )
))

# Configuração do layout do mapa
fig.update_layout(
    geo=dict(
        showland=True,
        landcolor='lightgray',
        countrycolor='white',
        coastlinecolor='white',
        projection_type='equirectangular'
    )
)

# Exibição do mapa
fig.show()

### Qual a média de propostas dos países?

### Experiencias Requeridas

In [13]:
exp = upwork['Experience Required'].dropna().reset_index()
exp = exp.groupby('Experience Required').size().reset_index().rename(columns={0:"# of Jobs"})
exp

,Experience Required,# of Jobs
0,Entry level,76
1,Expert,401
2,Intermediate,403


In [14]:
plot_graphics(exp, 
              "Experiência Requerida",
              "Experience Required", "# of Jobs")

In [3]:
upwork.head(1)

,Title,Description,Skills,Source Country,Client Porpouses,Average Hour Paid ($),Total Spent (K$),Domain of Job,Fixed Price ($),Price per Hour ($),Work on,Experience Required
0,Webcrawler to collect data of all escaperooms ...,For a specific project we need as much informa...,"['Data Scraping', 'Data Mining', 'Data Extract...",Netherlands,53,20.0,6.2,NaN,300.0,NaN,Remote Job,Expert


In [10]:
xp_paid = upwork[['Fixed Price ($)', 'Experience Required']].dropna().copy()
xp_paid.shape

(289, 2)

In [11]:
xp_paid = xp_paid.groupby(by='Experience Required')['Fixed Price ($)'].mean().reset_index()
xp_paid['Fixed Price ($)'] = xp_paid['Fixed Price ($)'].round(2)
plot_graphics(xp_paid, "Pagamento médio por experiência", "Experience Required", "Fixed Price ($)")

In [36]:
xp_skill = upwork[['Skills', 'Experience Required']].dropna().copy()
xp_skill['Skills'] = xp_skill['Skills'].apply(lambda row: eval(row))

In [37]:
s_x = []
for row in xp_skill.values:
    for s in row[0]:
        s_x.append((s, row[1]))

xp_skill = pd.DataFrame(s_x, columns=["Skill", "Experience"])
xp_skill.head()

,Skill,Experience
0,Data Scraping,Expert
1,Data Mining,Expert
2,Data Extraction,Expert
3,Company Research,Expert
4,Data Extraction,Expert


In [26]:
# levels = {"Expert":3, "Intermediate":2, "Entry level": 1}
# xp_skill['Experience'] = xp_skill['Experience'].replace(levels)
# xp_skill.head()

,Skill,Experience
0,Data Scraping,3
1,Data Mining,3
2,Data Extraction,3
3,Company Research,3
4,Data Extraction,3


In [42]:
exp = xp_skill.loc[xp_skill['Experience'] == "Expert"].groupby("Skill").size().reset_index().rename(columns={0:"# of Jobs"}).sort_values(by="# of Jobs", ascending=False)
inter = xp_skill.loc[xp_skill['Experience'] == "Intermediate"].groupby("Skill").size().reset_index().rename(columns={0:"# of Jobs"}).sort_values(by="# of Jobs", ascending=False)
entry = xp_skill.loc[xp_skill['Experience'] == "Entry level"].groupby("Skill").size().reset_index().rename(columns={0:"# of Jobs"}).sort_values(by="# of Jobs", ascending=False)
exp.head()

,Skill,# of Jobs
384,Python,173
303,Machine Learning,131
144,Data Analysis,120
64,Artificial Intelligence,86
165,Data Science,80


In [44]:
inter.head()

,Skill,# of Jobs
358,Python,158
121,Data Analysis,121
140,Data Scraping,94
289,Microsoft Excel,86
142,Data Visualization,83


In [65]:
def plot_invert_bar(data, title):
    fig = go.Figure(data=go.Bar(
        x=data['# of Jobs'],  # Eixo horizontal (experiência)
        y=data['Skill'],  # Eixo vertical (habilidades)
        orientation='h',  # Orientação horizontal das barras
    ))

    # Personalização do layout do gráfico
    fig.update_layout(
        font=dict(
            size=16  # Defina o tamanho da fonte desejado
        ),
        title=title,
        xaxis_title='# of Jobs',
        yaxis_title='Habilidades',
        width=500
    )

    # Exibição do gráfico
    fig.show()

In [66]:
plot_invert_bar(exp.head(10), "Expert")

In [67]:
plot_invert_bar(inter.head(10), "Intermediate")


In [68]:
plot_invert_bar(entry.head(10), "Entry Level")

## Análise das habilidades e áreas de interesse dos freelancers baseadas em grafo

In [8]:
freelas = pd.read_csv("../datasets/raw_99freelas_dataScience.csv")
freelas.head()

,name,skills,interest_area
0,Orlando H.,"['Excel Macros', 'Excel Recursos VB', 'Excel V...","['Desenvolvimento Web', 'Comercial', 'Teste de..."
1,Julio M.,"['Business Intelligence', 'PL/SQL', 'Aprendiza...","['Banco de Dados', 'Cloud Computing', 'Análise..."
2,Carlos E.,"['Aprendizado de Máquina (ML)', 'MySQL', 'Pyth...","['Banco de Dados', 'Desenvolvimento de Games',..."
3,Lucas P.,"['ABBYY FineReader', 'Framework JavaScript', '...","['Banco de Dados', 'Desenvolvimento de Games',..."
4,Jéssica,"['Android', 'CSS', 'Design de Website', 'HTML'...","['UX/UI & Web Design', 'Desenvolvimento Mobile..."


In [9]:
freelas['name'] = [i for i in range(len(freelas))]
freelas.head()

,name,skills,interest_area
0,0,"['Excel Macros', 'Excel Recursos VB', 'Excel V...","['Desenvolvimento Web', 'Comercial', 'Teste de..."
1,1,"['Business Intelligence', 'PL/SQL', 'Aprendiza...","['Banco de Dados', 'Cloud Computing', 'Análise..."
2,2,"['Aprendizado de Máquina (ML)', 'MySQL', 'Pyth...","['Banco de Dados', 'Desenvolvimento de Games',..."
3,3,"['ABBYY FineReader', 'Framework JavaScript', '...","['Banco de Dados', 'Desenvolvimento de Games',..."
4,4,"['Android', 'CSS', 'Design de Website', 'HTML'...","['UX/UI & Web Design', 'Desenvolvimento Mobile..."


#### Áreas de Interesse

In [10]:
interest_areas = []

for row in freelas['interest_area']:
    row = eval(row)
    row = [item.replace('Outra - ', '') for item in row]
    interest_areas += row

In [11]:
interest_areas = pd.DataFrame({
    "Interest Area": interest_areas
})
interest_areas.head()

,Interest Area
0,Desenvolvimento Web
1,Comercial
2,Teste de Software
3,"Web, Mobile & Software"
4,Análise de Dados & Estatística


In [12]:
interest_areas = interest_areas.groupby(by='Interest Area').size().reset_index().rename(columns={0:"Size"}).sort_values(by="Size", ascending=False)
interest_areas

,Interest Area,Size
7,Banco de Dados,116
16,Desenvolvimento Web,94
68,"Web, Mobile & Software",89
14,Desenvolvimento Desktop,81
2,Análise de Dados & Estatística,54
...,...,...
26,Empresarial,1
22,Diagramação,1
21,Design de Produto,1
19,Design 3D,1


### Áreas mais procuradas pelos freelancers

In [15]:
plot_graphics(interest_areas.head(20), "Áreas mais procuradas pelos Freelancers", 'Interest Area', 'Size')

### Skills

In [16]:
skills = []

for row in freelas['skills']:
    row = eval(row)
    row = [item.replace('Outra - ', '') for item in row]
    skills += row

skills = pd.DataFrame({
    "Skills": skills
})
skills.head()

skills = skills.groupby(by='Skills').size().reset_index().rename(columns={0:"Size"}).sort_values(by="Size", ascending=False)

In [20]:
plot_graphics(skills.head(50), "Skills mais populares entre os Freelancers", 'Skills', 'Size')

## Graph

In [21]:
skills = {skill:(i + 1) for i, skill in enumerate(skills['Skills'])}

skills

{'Python': 1,
 'Excel': 2,
 'Javascript': 3,
 'Mineração de Dados': 4,
 'Banco De Dados': 5,
 'Java': 6,
 'MySQL': 7,
 'SQL': 8,
 'Business Intelligence': 9,
 'PHP': 10,
 'ABBYY FineReader': 11,
 'Estatísticas': 12,
 'NodeJS': 13,
 'HTML5': 14,
 'PostgreSQL': 15,
 'CSS3': 16,
 'Análise de Projetos': 17,
 'Microsoft SQL Server': 18,
 'Armazenagem de Dados': 19,
 'Amazon Web Services': 20,
 'Google Analytics': 21,
 'Power BI\xa0': 22,
 'Android': 23,
 'Django': 24,
 'Desenvolvimento de API': 25,
 'Programação C#': 26,
 'AngularJS': 27,
 'Programação C++': 28,
 'ETL': 29,
 'Microsoft Office': 30,
 'NoSQL Couchbase': 31,
 'Entrada de Dados': 32,
 'Análise de Métricas': 33,
 'HTML': 34,
 'Linux': 35,
 'Análise de Dados': 36,
 'Gerenciamento de Projetos': 37,
 'Angular': 38,
 'Web Analytics': 39,
 'Mobile': 40,
 'Vue.JS': 41,
 '.NET Framework': 42,
 'Visual Basic': 43,
 'CSS': 44,
 'Administração': 45,
 'Wordpress': 46,
 'Adobe Photoshop': 47,
 'Inglês Avançado': 48,
 'Análise de Requisitos'

In [22]:
skills['None'] = 0

In [23]:
freelas.head()

,name,skills,interest_area
0,0,"['Excel Macros', 'Excel Recursos VB', 'Excel V...","['Desenvolvimento Web', 'Comercial', 'Teste de..."
1,1,"['Business Intelligence', 'PL/SQL', 'Aprendiza...","['Banco de Dados', 'Cloud Computing', 'Análise..."
2,2,"['Aprendizado de Máquina (ML)', 'MySQL', 'Pyth...","['Banco de Dados', 'Desenvolvimento de Games',..."
3,3,"['ABBYY FineReader', 'Framework JavaScript', '...","['Banco de Dados', 'Desenvolvimento de Games',..."
4,4,"['Android', 'CSS', 'Design de Website', 'HTML'...","['UX/UI & Web Design', 'Desenvolvimento Mobile..."


In [24]:
users = {}

for i, row in enumerate(freelas['skills']):
    v_vector = eval(row)
    v_vector = [skills[value] for value in v_vector]
    users[i] = v_vector

In [38]:
sizes = [len(list(users.values())[i]) for i in users.keys()]

In [43]:
sizes = np.sort(sizes)
users_idx = [i for i in range(len(sizes))]

fig = go.Figure([go.Scatter(
    x=users_idx,
    y=sizes,
    mode="markers+lines"
)])

fig.update_layout(
    title="Distribuição de quantidade de habilidades por usuário",
    xaxis_title="User ID",
    yaxis_title="# de habilidades",
    font=dict(
        size=16
    )
)

fig.show()

In [63]:
freelas_graph = {}

for user, u_skills in users.items():
    user_edges = []
    for user_2, u_skills_2 in users.items():
        if user == user_2: continue
        similarity_count = 0
        for s in u_skills:
            if s in u_skills_2:
                similarity_count += 1
                if similarity_count >= 3:
                    user_edges.append(user_2)
                    break

    freelas_graph[user] = user_edges

In [64]:
sizes = [len(list(freelas_graph.values())[i]) for i in freelas_graph.keys()]

sizes = np.sort(sizes)
users_idx = [i for i in range(len(sizes))]

fig = go.Figure([go.Scatter(
    x=users_idx,
    y=sizes,
    mode="markers+lines"
)])

fig.update_layout(
    title="Distribuição de conexões por usuário",
    xaxis_title="User ID",
    yaxis_title="# de conexões",
    font=dict(
        size=16
    )
)

fig.show()

Se eu usar o limite inferior de conexões muito alto, o número de conexões zera devido a falta de similaridade entre os vértices. Se eu usar muito baixo, o grafo se torna hiperdenso em relação as arestas. Mas se eu usar uma quantidade na média, terei uma perda de usuários os quais não se indentificam com muitos outros, porém terei uma topologia de grafo muito mais concisa. 

Portanto, defini empiricamente que 3 habilidades em comum significa que os usuários estão propensos a trabalhar na mesma área.

### Grafo Puro

In [67]:
print("Criando Grafo...")
graph = nx.from_dict_of_lists(freelas_graph)

# Obter as posições dos nós usando o algoritmo de layout spring
print("Obtendo posições dos nós...")
pos = nx.spring_layout(graph)

# Criar lista de posições x e y para os nós
x = [pos[node][0] for node in graph.nodes()]
y = [pos[node][1] for node in graph.nodes()]

print("Criando lista de arestas...")
# Criar lista de arestas
edge_trace = go.Scatter(
    x=[],
    y=[],
    line=dict(width=0.5, color='#888'),
    hoverinfo='none',
    mode='lines')

print("Adicionar arestas ao gráfico")
# Adicionar as arestas ao gráfico
for edge in graph.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_trace['x'] += tuple([x0, x1, None])
    edge_trace['y'] += tuple([y0, y1, None])

print("Criar lista de nós...")
# Criar lista de nós
node_trace = go.Scatter(
    x=x,
    y=y,
    text=list(graph.nodes()),
    mode='markers+text',
    hoverinfo='text',
    marker=dict(
        showscale=False,
        colorscale='YlGnBu',
        reversescale=True,
        color=[],
        size=10,
        colorbar=dict(
            thickness=15,
            title='Node Connections',
            xanchor='left',
            titleside='right'
        ),
        line_width=2))

# Adicionar a cor dos nós baseada no número de conexões
# node_trace.marker.color = [len(graph.neighbors(node)) for node in graph.nodes()]

print("Criar layout do gráfico...")
# Criar layout do gráfico
layout = go.Layout(
    title='Grafo',
    showlegend=False,
    hovermode='closest',
    margin=dict(b=20, l=5, r=5, t=40),
    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))

# Criar figura do Plotly
fig = go.Figure(data=[edge_trace, node_trace], layout=layout)

# Exibir o gráfico
fig.show()

Criando Grafo...
Obtendo posições dos nós...
Criando lista de arestas...
Adicionar arestas ao gráfico
Criar lista de nós...
Criar layout do gráfico...


### Topologia baseada em número de conexões

In [68]:
sizes = [len(list(freelas_graph.values())[i]) for i in freelas_graph.keys()]

In [70]:
print("Criando Grafo...")
graph = nx.from_dict_of_lists(freelas_graph)

# Obter as posições dos nós usando o algoritmo de layout spring
print("Obtendo posições dos nós...")
pos = nx.spring_layout(graph)

# Criar lista de posições x e y para os nós
x = [pos[node][0] for node in graph.nodes()]
y = [pos[node][1] for node in graph.nodes()]

print("Criando lista de arestas...")
# Criar lista de arestas
edge_trace = go.Scatter(
    x=[],
    y=[],
    line=dict(width=0.5, color='#888'),
    hoverinfo='none',
    mode='lines')

print("Adicionar arestas ao gráfico")
# Adicionar as arestas ao gráfico
for edge in graph.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_trace['x'] += tuple([x0, x1, None])
    edge_trace['y'] += tuple([y0, y1, None])

print("Criar lista de nós...")
# Criar lista de nós
node_trace = go.Scatter(
    x=x,
    y=y,
    # text=list(graph.nodes()),
    text=list(users.values()),
    mode='markers+text',
    hoverinfo='text',
    marker=dict(
        showscale=False,
        colorscale='YlGnBu',
        reversescale=True,
        color=[],
        size=sizes,
        colorbar=dict(
            thickness=15,
            title='Node Connections',
            xanchor='left',
            titleside='right'
        ),
        line_width=2))

# Adicionar a cor dos nós baseada no número de conexões
# node_trace.marker.color = [len(graph.neighbors(node)) for node in graph.nodes()]

print("Criar layout do gráfico...")
# Criar layout do gráfico
layout = go.Layout(
    title='Grafo',
    showlegend=False,
    hovermode='closest',
    margin=dict(b=20, l=5, r=5, t=40),
    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))

# Criar figura do Plotly
fig = go.Figure(data=[edge_trace, node_trace], layout=layout)

# Exibir o gráfico
fig.show()

Criando Grafo...
Obtendo posições dos nós...
Criando lista de arestas...
Adicionar arestas ao gráfico
Criar lista de nós...
Criar layout do gráfico...


Parte da densidade de conexões de alguns vértices se dão pelo fato de possuirem em seu currículo muitas habilidades diversas, o que o faz se conectar com uma maior quantidade de nós.

Seria interessante eu pegar um top k dos usuários com mais conexões e ver quais skills eles possuem.

In [87]:
users_idx = np.array(sizes)
users_idx = np.where(users_idx > 43)
users_idx[0]

array([ 28,  52,  86,  96, 165, 169, 224])

In [93]:
users_skills = [users[i] for i in users_idx[0]]
result_skills = []
for u_s in users_skills:
    result_skills.extend(u_s)

In [95]:
skills_vocab = {i:key for key,i in skills.items()}

In [99]:
skill_names = [skills_vocab[i] for i in result_skills]

most_interest_skills = pd.DataFrame({
    "skill":skill_names
})

most_interest_skills = most_interest_skills.groupby(by="skill").size().reset_index().rename(columns={0:"Size"}).sort_values(by="Size", ascending=False)
plot_graphics(most_interest_skills.head(10), "Habilidades comuns entre os vértices com maiores conexões", "skill", "Size")